In [ ]:
import json
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from llama_vis.tokenizer import Tokenizer, ChatFormat

In [10]:
with open('../experiments/llama3/arxiv/8b_walk_val_vis/2035_2.json') as f:
    result = json.load(f)

attention = np.load('../experiments/llama3/arxiv/8b_walk_val_vis/2035_2_attention.npy', allow_pickle=True)

In [ ]:
tokenizer = Tokenizer('../experiments/checkpoints/Meta-Llama-3-8B-Instruct-HF/original/tokenizer.model')
formatter = ChatFormat(tokenizer)
prompt_tokens = [formatter.encode_dialog_prompt(result['dialog'])]
prompt_tokens_text = [[tokenizer.model.decode_single_token_bytes(t) for t in toks] for toks in prompt_tokens]
out_tokens = [tokenizer.encode(result['result']['generation']['content'].strip(), bos=False, eos=False)]
out_tokens_text = [[tokenizer.model.decode_single_token_bytes(t) for t in toks] for toks in out_tokens]
tokens_text = [p + o for p, o in zip(prompt_tokens_text, out_tokens_text)]

In [12]:
cur_pos_list = list(attention.item().keys())
layer_idx_list = list(attention.item()[cur_pos_list[0]].keys())
head_idx_list = list(range(attention.item()[cur_pos_list[0]][layer_idx_list[0]].shape[0]))

In [ ]:
cur_pos_list

In [14]:
cur_pos = cur_pos_list[-2]
layer_idx = layer_idx_list[12]

norm = plt.Normalize(0, 1.4)

colored_text = ""

assert attention.item()[cur_pos_list[-1]][layer_idx][0][-1].shape[0] == len(tokens_text[0])

attention_scores = attention.item()[cur_pos][layer_idx].mean(0)[-1][365:2000]
tokens = tokens_text[0][365:2000]

attention_scores = np.concatenate((attention_scores[:126], np.asarray([0]), attention_scores[971:1177]))
tokens = tokens[:126] + [b' \\textit{[...]} '] + tokens[971:1177]

attention_scores = attention_scores / attention_scores.max()
for token, score in zip(tokens, attention_scores):
    if score < 0.01:
        r, g, b = 1, 1, 1
    else:
        r, g, b = matplotlib.cm.Oranges(norm(score))[:3]
    latex_color = f"{int(r * 255)},{int(g * 255)},{int(b * 255)}"
    colored_text += (f"\\cctext{{{latex_color}}}{{{token.decode('utf-8')}}}")

In [ ]:
print(prompt_tokens_text)
colored_text = colored_text.replace('_', '\_')
colored_text = colored_text.replace('$', '\\$')
colored_text = colored_text.replace('$\\', '$')
colored_text = '\\\\'.join(colored_text.splitlines())
print(colored_text)